In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim


from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures





In [11]:

class Agent(nn.Module):
    def __init__(self, num_node_features,hidden_channels1,hidden_channels2):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels1)
        self.conv2 = GCNConv( hidden_channels1,hidden_channels2)
        self.lin1 = nn.Linear(hidden_channels2, 64)
        self.lin2 = nn.Linear(64, 10)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index  # x: [2708, 1433]

        x = self.conv1(x, edge_index)            # [2708, hidden1]
        x = F.relu(x)
        x = self.conv2(x, edge_index)            # [2708, hidden2]
        x = F.relu(x)

        x = self.lin1(x)                          # [2708, 64]
        x = F.relu(x)
        x = self.lin2(x)                          # [2708, fc_out]

        x = F.tanh(x)
        return x


In [12]:

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset[0]


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [13]:
dataset[0].edge_index[:,7]

tensor([332,   2])

In [26]:

model = Agent(num_node_features=dataset.num_node_features,hidden_channels1=16,hidden_channels2=32)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data = dataset[0]  
out = model(data)
o = out[1]
mapped = torch.where(o > 0.5, 1,
           torch.where(o < -0.5, -1, 0))

mapped

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])